<a href="https://colab.research.google.com/github/veiro/ADS2020/blob/master/imputacion/4_GAIN_with_covid_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo:
Probar GAIN para imputar datos en el juego de datos de Covid.

# Trabajo en siguiente iteracion
La salida de este notebook es un set de datos con datos imputados llamado dataset-GAIN.

# Configuracion ambiente

In [1]:
import os
import sys

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import pandas
from sklearn.model_selection import train_test_split
from google.colab import drive

from datetime import datetime
#from pycaret.regression import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
USAR_GOOGLE_DRIVE =  True
MODO_DESARROLLO = False

# Preprocesamiento datos

In [3]:
if (USAR_GOOGLE_DRIVE):
  drive.mount('/content/gdrive')
  PATH_DATA_PROCESADA = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/data/data-procesada"
  PATH_IMPUTATION = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/imputacion/GAIN/"
else:
  PATH_DATA_PROCESADA = "data/data-procesada"


Mounted at /content/gdrive


In [4]:
%%time
data_filepath = PATH_DATA_PROCESADA+'/csse_covid_19_data.csv'
data_full = pandas.read_csv(data_filepath)


CPU times: user 6.4 s, sys: 1.28 s, total: 7.68 s
Wall time: 9.67 s


In [5]:
%%time
data_sin_columnas_completas = data_full.drop(['FIPS', 'Admin2', 'Recovered', 'Active', 'Combined_Key'], axis=1)
data_sin_null = data_sin_columnas_completas.dropna(subset=['Province_State', 'Lat', 'Long_', 'Incident_Rate', 'Case_Fatality_Ratio'])


if(MODO_DESARROLLO):
   data_utilizar = data_sin_null[:10000]
else:
  data_utilizar = data_sin_null


CPU times: user 732 ms, sys: 180 ms, total: 912 ms
Wall time: 926 ms


In [6]:
# variables gloables
encoded_df_Country_Region_cols = 0
encoderOHE = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoderLE1 = LabelEncoder()
encoderLE2 = LabelEncoder()
encoderLE3 = LabelEncoder()

In [7]:

# ENCODING
# 1- onehotenconding de Country_Region
df=data_utilizar

encoded_data = encoderOHE.fit_transform( df[['Country_Region']])
encoded_df_Country_Region = pandas.DataFrame(encoded_data, columns=encoderOHE.get_feature_names_out(['Country_Region']))
encoded_df_Country_Region_cols = encoded_df_Country_Region.shape[1]

df = df.drop(['Country_Region'], axis=1)

# hice este concat horrible por que por alguna razon no me andaba bien el concant ordinario
pos = encoded_df_Country_Region_cols
for col in df.columns:
  encoded_df_Country_Region.insert( pos, col, df[col].to_numpy(), True)
  pos=pos+1

df = encoded_df_Country_Region

# 2 Province_State, Last_Update, date
df['Province_State_encoded'] = encoderLE1.fit_transform(df['Province_State'])
df = df.drop(['Province_State'], axis=1)

df['Last_Update_encoded'] = encoderLE2.fit_transform(df['Last_Update'])
df = df.drop(['Last_Update'], axis=1)

df['date_encoded'] = encoderLE3.fit_transform(df['date'])
df = df.drop(['date'], axis=1)
  #return df,  encoded_df_Country_Region_cols

data_transformed=df



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# GAIN


In [8]:
ruta_req= PATH_IMPUTATION+'requirements.txt'
!pip install -r $ruta_req

In [9]:
# Necessary packages
import numpy as np
from keras.datasets import mnist
import numpy as np
#import tensorflow as tf
##IF USING TF 2 use following import to still use TF < 2.0 Functionalities
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import numpy as np
from tqdm import tqdm


Instructions for updating:
non-resource variables are not supported in the long term


## utils


In [10]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.

  Args:
    - data: original data

  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)

    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}

  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters


def renormalization (norm_data, norm_parameters):
  '''Renormalize data from [0, 1] range to the original range.

  Args:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization

  Returns:
    - renorm_data: renormalized original data
  '''

  min_val = norm_parameters['min_val']
  max_val = norm_parameters['max_val']

  _, dim = norm_data.shape
  renorm_data = norm_data.copy()

  for i in range(dim):
    renorm_data[:,i] = renorm_data[:,i] * (max_val[i] + 1e-6)
    renorm_data[:,i] = renorm_data[:,i] + min_val[i]

  return renorm_data


def rounding (imputed_data, data_x):
  '''Round imputed data for categorical variables.

  Args:
    - imputed_data: imputed data
    - data_x: original data with missing values

  Returns:
    - rounded_data: rounded imputed data
  '''

  _, dim = data_x.shape
  rounded_data = imputed_data.copy()

  for i in range(dim):
    temp = data_x[~np.isnan(data_x[:, i]), i]
    # Only for the categorical variable
    if len(np.unique(temp)) < 20:
      rounded_data[:, i] = np.round(rounded_data[:, i])

  return rounded_data

# deprecada asi utiliza la misma que MIDA
def rmse_loss (ori_data, imputed_data, data_m):
  '''Compute RMSE loss between ori_data and imputed_data

  Args:
    - ori_data: original data without missing values
    - imputed_data: imputed data
    - data_m: indicator matrix for missingness

  Returns:
    - rmse: Root Mean Squared Error
  '''

  ori_data, norm_parameters = normalization(ori_data)
  imputed_data, _ = normalization(imputed_data, norm_parameters)

  # Only for missing values
  nominator = np.sum(((1-data_m) * ori_data - (1-data_m) * imputed_data)**2)
  denominator = np.sum(1-data_m)

  rmse = np.sqrt(nominator/float(denominator))

  return rmse


from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse_loss_v2 (ori_data, imputed_data, data_m):
  '''Compute RMSE loss between ori_data and imputed_data

  Args:
    - ori_data: original data without missing values
    - imputed_data: imputed data
    - data_m: indicator matrix for missingness

  Returns:
    - rmse: Root Mean Squared Error
  '''
  rmse_sum = 0
  ori_data, norm_parameters = normalization(ori_data)
  imputed_data, _ = normalization(imputed_data, norm_parameters)
  rows, cols = ori_data.shape
  for i in range(cols) :
    if ori_data[:,i].sum() > 0 :
        y_actual = ori_data[:,i][data_m[:,i]]
        y_predicted = imputed_data[:,i][data_m[:,i]]

        rmse = sqrt(mean_squared_error(y_actual, y_predicted))
        rmse_sum += rmse

  return rmse_sum


def xavier_init(size):
  '''Xavier initialization.

  Args:
    - size: vector size

  Returns:
    - initialized random vector.
  '''
  in_dim = size[0]
  xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
  return tf.random_normal(shape = size, stddev = xavier_stddev)


def binary_sampler(p, rows, cols):
  '''Sample binary random variables.

  Args:
    - p: probability of 1
    - rows: the number of rows
    - cols: the number of columns

  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = [rows, cols])
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix


def uniform_sampler(low, high, rows, cols):
  '''Sample uniform random variables.

  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns

  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = [rows, cols])


def sample_batch_index(total, batch_size):
  '''Sample index of the mini-batch.

  Args:
    - total: total number of samples
    - batch_size: batch size

  Returns:
    - batch_idx: batch index
  '''
  total_idx = np.random.permutation(total)
  batch_idx = total_idx[:batch_size]
  return batch_idx

## data loader

In [11]:
def data_loader (data_name, miss_rate):
  '''Loads datasets and introduce missingness.

  Args:
    - data_name: letter, spam, or mnist
    - miss_rate: the probability of missing components

  Returns:
    data_x: original data
    miss_data_x: data with missing values
    data_m: indicator matrix for missing components
  '''

  # Load data
  if data_name in ['letter', 'spam']:
    file_name = PATH_IMPUTATION + '/data/'+data_name+'.csv'
    data_x = np.loadtxt(file_name, delimiter=",", skiprows=1)
  elif data_name == 'mnist':
    (data_x, _), _ = mnist.load_data()
    data_x = np.reshape(np.asarray(data_x), [60000, 28*28]).astype(float)
  elif data_name == 'covid':
    print("Proceso covid dataset")
    data_x = data_transformed.to_numpy()  #np.loadtxt(file_name, delimiter=",", skiprows=1)
  # Parameters
  no, dim = data_x.shape

  # Introduce missing data
  data_m = binary_sampler(1-miss_rate, no, dim)
  miss_data_x = data_x.copy()
  miss_data_x[data_m == 0] = np.nan

  return data_x, miss_data_x, data_m

## **GAIN**


In [12]:

def gain (data_x, gain_parameters):
  '''Impute missing values in data_x

  Args:
    - data_x: original data with missing values
    - gain_parameters: GAIN network parameters:
      - batch_size: Batch size
      - hint_rate: Hint rate
      - alpha: Hyperparameter
      - iterations: Iterations

  Returns:
    - imputed_data: imputed data
  '''
  # Define mask matrix
  data_m = 1-pandas.isna(data_x)
  # System parameters
  batch_size = gain_parameters['batch_size']
  hint_rate = gain_parameters['hint_rate']
  alpha = gain_parameters['alpha']
  iterations = gain_parameters['iterations']

  # Other parameters
  no, dim = data_x.shape

  # Hidden state dimensions
  h_dim = int(dim)

  # Normalization

  norm_data, norm_parameters = normalization(data_x)
  norm_data_x = np.nan_to_num(norm_data, 0)

  ## GAIN architecture
  # Input placeholders
  # Data vector
  X = tf.placeholder(tf.float32, shape = [None, dim])
  # Mask vector
  M = tf.placeholder(tf.float32, shape = [None, dim])
  # Hint vector
  H = tf.placeholder(tf.float32, shape = [None, dim])

  # Discriminator variables
  D_W1 = tf.Variable(xavier_init([dim*2, h_dim])) # Data + Hint as inputs
  D_b1 = tf.Variable(tf.zeros(shape = [h_dim]))

  D_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
  D_b2 = tf.Variable(tf.zeros(shape = [h_dim]))

  D_W3 = tf.Variable(xavier_init([h_dim, dim]))
  D_b3 = tf.Variable(tf.zeros(shape = [dim]))  # Multi-variate outputs

  theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]

  #Generator variables
  # Data + Mask as inputs (Random noise is in missing components)
  G_W1 = tf.Variable(xavier_init([dim*2, h_dim]))
  G_b1 = tf.Variable(tf.zeros(shape = [h_dim]))

  G_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
  G_b2 = tf.Variable(tf.zeros(shape = [h_dim]))

  G_W3 = tf.Variable(xavier_init([h_dim, dim]))
  G_b3 = tf.Variable(tf.zeros(shape = [dim]))

  theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]

  ## GAIN functions
  # Generator
  def generator(x,m):
    # Concatenate Mask and Data
    inputs = tf.concat(values = [x, m], axis = 1)
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)
    # MinMax normalized output
    G_prob = tf.nn.sigmoid(tf.matmul(G_h2, G_W3) + G_b3)
    return G_prob

  # Discriminator
  def discriminator(x, h):
    # Concatenate Data and Hint
    inputs = tf.concat(values = [x, h], axis = 1)
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
    D_logit = tf.matmul(D_h2, D_W3) + D_b3
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob

  ## GAIN structure
  # Generator
  G_sample = generator(X, M)

  # Combine with observed data
  Hat_X = X * M + G_sample * (1-M)

  # Discriminator
  D_prob = discriminator(Hat_X, H)

  ## GAIN loss
  D_loss_temp = -tf.reduce_mean(M * tf.log(D_prob + 1e-8) \
                                + (1-M) * tf.log(1. - D_prob + 1e-8))

  G_loss_temp = -tf.reduce_mean((1-M) * tf.log(D_prob + 1e-8))

  MSE_loss = \
  tf.reduce_mean((M * X - M * G_sample)**2) / tf.reduce_mean(M)

  D_loss = D_loss_temp
  G_loss = G_loss_temp + alpha * MSE_loss

  ## GAIN solver
  D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
  G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

  ## Iterations
  sess = tf.Session()
  sess.run(tf.global_variables_initializer())

  # Start Iterations
  for it in tqdm(range(iterations)):

    # Sample batch
    batch_idx = sample_batch_index(no, batch_size)
    X_mb = norm_data_x[batch_idx, :]
    M_mb = data_m[batch_idx, :]
    # Sample random vectors
    Z_mb = uniform_sampler(0, 0.01, batch_size, dim)
    # Sample hint vectors
    H_mb_temp = binary_sampler(hint_rate, batch_size, dim)
    H_mb = M_mb * H_mb_temp

    # Combine random vectors with observed vectors
    X_mb = M_mb * X_mb + (1-M_mb) * Z_mb

    _, D_loss_curr = sess.run([D_solver, D_loss_temp],
                              feed_dict = {M: M_mb, X: X_mb, H: H_mb})
    _, G_loss_curr, MSE_loss_curr = \
    sess.run([G_solver, G_loss_temp, MSE_loss],
             feed_dict = {X: X_mb, M: M_mb, H: H_mb})

  ## Return imputed data
  Z_mb = uniform_sampler(0, 0.01, no, dim)
  M_mb = data_m
  X_mb = norm_data_x
  X_mb = M_mb * X_mb + (1-M_mb) * Z_mb

  imputed_data = sess.run([G_sample], feed_dict = {X: X_mb, M: M_mb})[0]

  imputed_data = data_m * norm_data_x + (1-data_m) * imputed_data

  # Renormalization
  imputed_data = renormalization(imputed_data, norm_parameters)

  # Rounding
  imputed_data = rounding(imputed_data, data_x)

  return imputed_data

## main

In [13]:
%%time
def main (data_name,miss_rate= 0.2, batch_size=128, hint_rate=0.9, alpha= 100,iterations=10000):
  data_name = data_name
  miss_rate = miss_rate

  gain_parameters = {'batch_size': batch_size,
                      'hint_rate': hint_rate,
                      'alpha': alpha,
                      'iterations': iterations}
  ori_data_x, miss_data_x, data_m = data_loader(data_name, miss_rate)
  imputed_data_x = gain(miss_data_x, gain_parameters)
  rmse = rmse_loss_v2 (ori_data_x, imputed_data_x, data_m)
  return imputed_data_x, rmse

imputed_data_x, rmse = main(data_name='covid',miss_rate= 0.2, batch_size=128, hint_rate=0.9, alpha= 100,iterations=10000)
print('rmse', rmse)


Proceso covid dataset


100%|██████████| 10000/10000 [20:30<00:00,  8.13it/s]


rmse 1.6936092995032181
CPU times: user 21min 45s, sys: 21.3 s, total: 22min 7s
Wall time: 21min 29s


# Guardar resultados

In [14]:
df = pandas.DataFrame(imputed_data_x, columns = data_transformed.columns)
df.head(5)

,Country_Region_Australia,Country_Region_Belgium,Country_Region_Brazil,Country_Region_Canada,Country_Region_Chile,Country_Region_China,Country_Region_Colombia,Country_Region_Denmark,Country_Region_France,Country_Region_Germany,...,Country_Region_United Kingdom,Lat,Long_,Confirmed,Deaths,Incident_Rate,Case_Fatality_Ratio,Province_State_encoded,Last_Update_encoded,date_encoded
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,26.368495,149.012400,118.0,3.000000,4114.576342,2.542373,42.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-33.868800,-72.393774,4947.0,54.000000,60.938655,1.091571,350.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-12.463400,130.845600,75.0,823.745321,30.537459,0.000000,368.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-27.469800,153.025100,1255.0,6.000000,24.533281,1.130097,426.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,26.772760,138.600700,580.0,1109.335801,33.020211,0.689655,488.0,1.0,0.0


In [15]:
df.shape

(2992223, 34)

In [16]:
%%time
# DECODE
# 1- onehotenconding de Country_Region.
encoded_df_Country_Region = df.iloc[:,  :encoded_df_Country_Region_cols]
df['Country_Region'] = encoderOHE.inverse_transform(encoded_df_Country_Region)
df.drop(columns=df.columns[:encoded_df_Country_Region_cols], axis=1,  inplace=True)



#2- Province_State, Last_Update, date
Province_State_encoded_unique = data_transformed.Province_State_encoded.unique().tolist()
for i,  row in df.iterrows():
  if( row['Province_State_encoded'] not in Province_State_encoded_unique):
    rounded = round(row['Province_State_encoded'],0)
    df.at[i,'Province_State_encoded'] = rounded
df['Province_State'] = encoderLE1.inverse_transform(df['Province_State_encoded'].astype(int) )


Last_Update_encoded_unique = data_transformed.Last_Update_encoded.unique().tolist()
for i,  row in df.iterrows():
  if( row['Last_Update_encoded'] not in Last_Update_encoded_unique):
    rounded = round(row['Last_Update_encoded'],0)
    df.at[i,'Last_Update_encoded'] = rounded
    #print(row['Last_Update_encoded'])
df['Last_Update'] = encoderLE2.inverse_transform(df['Last_Update_encoded'].astype(int) )

date_encoded_unique = data_transformed.date_encoded.unique().tolist()
for i,  row in df.iterrows():
  if( row['date_encoded'] not in date_encoded_unique):
    rounded = round(row['date_encoded'],0)
    df.at[i,'date_encoded'] = rounded
    #print(row['date_encoded'])
df['date'] = encoderLE3.inverse_transform(df['date_encoded'].astype(int) )

df = df.drop(['Province_State_encoded'], axis=1)
df = df.drop(['Last_Update_encoded'], axis=1)
df = df.drop(['date_encoded'], axis=1)

#3- ordeno columnas

col = df.pop('Province_State')
df.insert(0, col.name, col)

col = df.pop('Country_Region')
df.insert(1,  col.name, col)

col = df.pop('Last_Update')
df.insert(2,  col.name, col)

# cambio tipo de dato

df['Confirmed'] = df['Confirmed'].astype(int)

df['Deaths'] = df['Deaths'].astype(int)


# cantidad de cifras significativas>

df["Lat"] = round(df['Lat'],6)
df["Long_"] = round(df['Long_'],4)



CPU times: user 10min 39s, sys: 4.43 s, total: 10min 44s
Wall time: 10min 45s


In [17]:
df.shape

(2992223, 10)

In [18]:
# al imputar alguna de las 25 columnas en las que se separo Country_Region lo hizo mal 284 veces.
df.isna().sum()

Province_State             0
Country_Region         77172
Last_Update                0
Lat                        0
Long_                      0
Confirmed                  0
Deaths                     0
Incident_Rate              0
Case_Fatality_Ratio        0
date                       0
dtype: int64

In [19]:
# reseteo indices
data_utilizar = data_utilizar.reset_index(drop=True)
df = df.reset_index(drop=True)

diferrence = data_utilizar.compare(df)

In [20]:
# esto es para ver que tan bien estuvo, NaN significa que son iguales.
diferrence

Province_State        Country_Region                Last_Update  \
                  self other            self other                 self   
0                  NaN   NaN             NaN   NaN                  NaN   
1                  NaN   NaN       Australia  None                  NaN   
2                  NaN   NaN             NaN   NaN                  NaN   
3                  NaN   NaN             NaN   NaN                  NaN   
4                  NaN   NaN             NaN   NaN                  NaN   
...                ...   ...             ...   ...                  ...   
2992218            NaN   NaN  United Kingdom    US                  NaN   
2992219            NaN   NaN  United Kingdom  None  2023-03-10 04:21:03   
2992220            NaN   NaN             NaN   NaN                  NaN   
2992221            NaN   NaN  United Kingdom  None                  NaN   
2992222            NaN   NaN             NaN   NaN                  NaN   

                                  Lat                Long_           \
                       other     self      other      self    other   
0                        NaN -35.4735  26.368495       NaN      NaN   
1                        NaN      NaN        NaN  151.2093 -72.3938   
2                        NaN      NaN        NaN       NaN      NaN   
3                        NaN      NaN        NaN       NaN      NaN   
4                        NaN -34.9285  26.772760       NaN      NaN   
...                      ...      ...        ...       ...      ...   
2992218                  NaN      NaN        NaN       NaN      NaN   
2992219  2022-01-10 04:22:07      NaN        NaN       NaN      NaN   
2992220                  NaN      NaN        NaN       NaN      NaN   
2992221                  NaN      NaN        NaN       NaN      NaN   
2992222                  NaN      NaN        NaN       NaN      NaN   

         Confirmed             Deaths         Incident_Rate               \
              self     other     self   other          self        other   
0              NaN       NaN      NaN     NaN     27.563653  4114.576342   
1              NaN       NaN      NaN     NaN           NaN          NaN   
2              NaN       NaN      0.0   823.0           NaN          NaN   
3              NaN       NaN      NaN     NaN           NaN          NaN   
4              NaN       NaN      4.0  1109.0           NaN          NaN   
...            ...       ...      ...     ...           ...          ...   
2992218        NaN       NaN      0.0   426.0           NaN          NaN   
2992219     2166.0   36575.0      NaN     NaN           NaN          NaN   
2992220  2157720.0  904641.0  16806.0  9638.0           NaN          NaN   
2992221        NaN       NaN      NaN     NaN           NaN          NaN   
2992222        NaN       NaN  11500.0  5970.0           NaN          NaN   

        Case_Fatality_Ratio                  date              
                       self     other        self       other  
0                       NaN       NaN         NaN         NaN  
1                       NaN       NaN         NaN         NaN  
2                       NaN       NaN         NaN         NaN  
3                  0.478088  1.130097         NaN         NaN  
4                       NaN       NaN         NaN         NaN  
...                     ...       ...         ...         ...  
2992218                 NaN       NaN  03-09-2023  06-05-2021  
2992219                 NaN       NaN         NaN         NaN  
2992220            0.778878  0.014509         NaN         NaN  
2992221                 NaN       NaN         NaN         NaN  
2992222            1.280455  0.015473         NaN         NaN  

[2971510 rows x 20 columns]

In [21]:
if (USAR_GOOGLE_DRIVE):
  df.to_csv(PATH_DATA_PROCESADA+'/csse_covid_19_data_GAIN._Desarrollo='+str(MODO_DESARROLLO)+'.csv', index=False)